In [6]:
# Run setup_imports to enable importing from src/
%run ./setup_imports.py

In [7]:
from qonscious.constraints.packed_chsh import PackedCHSHTest
from qonscious.policies import MinimumAcceptableValue
from qonscious.core.executor import run_conditionally
from qonscious.core.aer_sampler_adapter import AerSamplerAdapter

In [8]:
passing_policy = MinimumAcceptableValue(1.5)
failing_policy = MinimumAcceptableValue(2.9)

In [9]:
def on_pass(backend, introspection):
    print("CHSH test passed!")
    print(f"Score: {introspection['CHSH_score']:.3f}")
    return introspection

def on_fail(backend, introspection):
    print("CHSH test failed.")
    print(f"Score: {introspection['CHSH_score']:.3f}")
    return introspection

In [10]:
backend = AerSamplerAdapter()
constraint = PackedCHSHTest(passing_policy)

print("Running CHSH test with passing threshold...\n")
run_conditionally(
    backend_adapter=backend,
    constraint=constraint,
    on_pass=on_pass,
    on_fail=on_fail,
    shots=2048
)


Running CHSH test with passing threshold...

CHSH test passed!
Score: 2.802


/var/folders/sy/b2bz14411yv4n78xk1t7k31r0000gn/T/ipykernel_3299/3043173648.py:1: DeprecationWarning: Sampler has been deprecated as of Aer 0.15, please use SamplerV2 instead.
  backend = AerSamplerAdapter()


{'E00': 0.70703125,
 'E01': 0.701171875,
 'E10': 0.69140625,
 'E11': -0.7021484375,
 'CHSH_score': 2.8017578125}

In [11]:
constraint = PackedCHSHTest(failing_policy)

print("\nRunning CHSH test with failing threshold...\n")
run_conditionally(
    backend_adapter=backend,
    constraint=constraint,
    on_pass=on_pass,
    on_fail=on_fail,
    shots=2048
)


Running CHSH test with failing threshold...

CHSH test failed.
Score: 2.865


{'E00': 0.728515625,
 'E01': 0.6953125,
 'E10': 0.712890625,
 'E11': -0.728515625,
 'CHSH_score': 2.865234375}